Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [130]:
NAME = "Aseem Sachdeva"

# Presenting Uncertainty
## School of Information, University of Michigan

## Week 2: Assignment Overview
Version 1.1
### The objectives for this week are for you to:

- review the concept of standard error
- construct a confidence distribution
- use a confidence distribution to construct a density plot, interval plot, CCDF barplot, and quantile dotplot
- apply one or more of the above techniques to the World Happiness dataset

In [131]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
import random
from scipy import stats
from scipy.stats import norm

# Part 1. Standard error and confidence distributions (14 points)

## 1.1 Bootstrap sampling distribution of phone pickups example

Recall in assignment 1 we used a sample from a hypothetical dataset of counts of how many times students at a school picked up their phones. Let's generate such a hypothetical dataset again. We'll make a sample of 20 students and the number of times each student picked up their phone:

In [132]:
np.random.seed(1234)   # for reproducibility

pickups_df = pd.DataFrame({"pickups": np.random.poisson(lam=40, size=20)})

pickups_df.describe()

,pickups
count,20.000000
mean,39.350000
std,6.072154
min,27.000000
25%,36.750000
50%,38.500000
75%,43.250000
max,52.000000


In [133]:
pickups_df

,pickups
0,34
1,39
2,46
3,36
4,37
5,43
6,37
7,40
8,37
9,29


### Question 1.1.1 (2 points)

Using what you learned in assignment 1, generate a bootstrap sampling distribution of the mean number of pickups and assign it to the variable `bootstrap_means`:

In [134]:
B = 5000
np.random.seed(1234) # for reproducibility
sample_pickups = pickups_df['pickups']

# YOUR CODE HERE
bootstrap_means = []
Replications = np.array([np.random.choice(sample_pickups, size=20, replace = True) for i in range(B)])
bootstrap_means = np.mean(Replications, axis=1)

In [135]:
# the bootstrap sampling mean should be close to the sample mean
sample_mean = pickups_df['pickups'].mean()
sample_mean_se = np.std(pickups_df['pickups'], ddof=1) / np.sqrt(len(pickups_df))
assert len(bootstrap_means) == 5000, "Bootstrap sampling distribution: length of bootstrap_means should be 5000"
assert np.abs(np.std(bootstrap_means) - sample_mean_se) / sample_mean_se < 0.05, "Bootstrap sampling distribution: SD of boostrap_means does not match SE of sample_pickups"
assert np.abs(np.mean(bootstrap_means) - sample_mean)/sample_mean < 5*1e-3, "Bootstrap sampling distribution: mean of bootstrap_means does not match sample_mean"

### Question 1.1.2 (2 points)

Visualize the bootstrap sampling distribution of the mean as a density plot.

Hint: see assignment 1.

In [136]:
# YOUR CODE HERE
df = pd.DataFrame(data = {"bootstrap_means" : bootstrap_means})

pop_density_chart_bootstrap = alt.Chart(df).transform_density(
    density='bootstrap_means',
    as_=['bootstrap_means', 'density'],
    
).mark_area(color="lightgray").encode(
    x=alt.X('bootstrap_means', axis=alt.Axis(title="Bootstrap distribution of phone pickups")),
    y=alt.Y('density:Q', axis=alt.Axis(title="Density"))
)

pop_mean_chart_bootstrap = alt.Chart(df).mark_rule(color="black", strokeDash=[1,1]).encode(
    x="mean(bootstrap_means)"
)

pop_chart_bootstrap = pop_density_chart_bootstrap + pop_mean_chart_bootstrap

pop_chart_bootstrap

alt.LayerChart(...)

## 1.2 Standard error and Normal sampling distribution for mean pickups

The sampling distribution of a mean approaches the shape of a Normal distribution as the sample size increases. Instead of using bootstrapping to derive the distribution, we could instead use the *standard error*, which is an estimate of the standard deviation of the sampling distribution, assuming the sampling distribution is Normal.

The standard error of the mean ($s_\bar{x}$) is calculated using the sample standard deviation ($s$) and the sample size ($n$):

$s_\bar{x} = \frac{s}{\sqrt{n}}$

This calculation can be done as follows:

In [137]:
# manual calculation of standard error
pickups_se = pickups_df['pickups'].std(ddof=1) / np.sqrt(len(pickups_df))
print("Manual calculation of SE:           ", pickups_se)

# or using scipy.stats.sem() (should be the same)
pickups_se = stats.sem(pickups_df['pickups'])
print("scipy.stats.sem() calculation of SE:", pickups_se)


Manual calculation of SE:            1.357774882511437
scipy.stats.sem() calculation of SE: 1.357774882511437


Given the sample mean pickups ($\bar{x}$ = `pickups_mean`) and the sample mean standard error ($s_\bar{x}$ = `pickups_se`), we can construct a sampling distribution using the standard error. As discussed in lecture, there are three functions we'll want to define in order to construct different uncertainty visualizations: the probability density function (PDF), the cumulative distribution function (CDF), and the quantile function (also called the percentage points function, PPF). For a mean and standard error, these are:

$\begin{align}
\mathrm{density~function~(PDF):~} & f_\mathrm{Normal}\left(x \middle| \bar{x}, s_\bar{x}\right) & \mathtt{stats.norm.pdf()}\\
\mathrm{cumulative~distribution~function~(CDF):~} & F_\mathrm{Normal}\left(x \middle| \bar{x}, s_\bar{x}\right) & \mathtt{stats.norm.cdf()}\\
\mathrm{quantile~function~(PPF):~} & F^{-1}_\mathrm{Normal}\left(p \middle| \bar{x}, s_\bar{x}\right) & \mathtt{stats.norm.ppf()}
\end{align}$

In [138]:
pickups_mean = pickups_df['pickups'].mean()

# generate evenly-spaced numbers covering the range of values of pickups
# that we will calculate the density of the sampling distribution at
x = np.linspace(
    start = pickups_df['pickups'].min(), 
    stop = pickups_df['pickups'].max(),
    num = 1001
)

# this is the density of the sampling distribution: f_Normal(x | x_bar, se)
density = stats.norm.pdf(x, pickups_mean, pickups_se)

df = pd.DataFrame({"Normal sampling distribution of mean pickups": x, "density": density})

pickups_se_chart = alt.Chart(df).mark_line().encode(
    x="Normal sampling distribution of mean pickups",
    y="density"
)

pickups_se_chart

alt.Chart(...)

## 1.3 t-based confidence distribution for mean pickups

A slight improvement to the sampling distribution above is to use a Student's t confidence distribution. At small sample sizes, the Normal approximation is less accurate; a scaled-and-shifted Student's t distribution will have slightly fatter tails than the Normal distribution. 

To determine how fat the tails are, the t distribution uses a *degrees of freedom* parameter, which for the estimate of a single independent mean of sample size $n$ is equal to $n - 1$. Thus, an improved confidence distribution over the Normal approximation above is:

$\begin{align}
\mathrm{density~function~(PDF):~} & f_\mathrm{t}\left(x \middle| n - 1, \bar{x}, s_\bar{x}\right) & \mathtt{stats.t.pdf()}\\
\mathrm{cumulative~distribution~function~(CDF):~} & F_\mathrm{t}\left(x \middle| n - 1, \bar{x}, s_\bar{x}\right) & \mathtt{stats.t.cdf()}\\
\mathrm{quantile~function~(PPF):~} & F^{-1}_\mathrm{t}\left(p \middle| n - 1, \bar{x}, s_\bar{x}\right) & \mathtt{stats.t.ppf()}
\end{align}$

This confidence distribution is related to the common Student's t test: confidence intervals from this distribution are the same as the confidence intervals you would generate from a t test.

### Question 1.3.1 Visualize the Student's t confidence distribution for the mean (5 points)

Visualize the density of the t-based confidence distribution described above in a similar manner to how the Normal sampling distribution was visualized, but use a different color for the line (this will help you with the next question):


In [139]:
# YOUR CODE HERE


x = np.linspace(
    start = pickups_df['pickups'].min(), 
    stop = pickups_df['pickups'].max(),
    num = 1001
)


density_t =  stats.t.pdf(x, (len(pickups_df)-1), pickups_mean, pickups_se)

df = pd.DataFrame({"Student t-based confidence distribution of pickups": x, "density": density_t})

pickups_t_chart = alt.Chart(df).mark_line(color='orange').encode(
    x="Student t-based confidence distribution of pickups",
    y="density"
)

pickups_t_chart

alt.Chart(...)

### Question 1.3.2 Visualize and compare uncertainty distributions (5 points)

Visualize all three distributions you have constructed so far in a single plot. 

Hint: if you have saved your previous plots in three separate variables and made them with different enough encodings that they can be distinguished by eye, this answer can be as simple as `pickups_bootstrap_chart + pickups_se_chart + pickups_t_chart`

In [140]:
#pop_chart_bootstrap(redeclaring the code with different title so it will show in combined chart)
df = pd.DataFrame(data = {"bootstrap_means" : bootstrap_means})

pop_density_chart_bootstrap = alt.Chart(df).transform_density(
    density='bootstrap_means',
    as_=['bootstrap_means', 'density'],
    
).mark_area(color="lightgray").encode(
    x=alt.X('bootstrap_means', axis=alt.Axis(title="Combined distribution of phone pickups")),
    y=alt.Y('density:Q', axis=alt.Axis(title="Density"))
)

pop_mean_chart_bootstrap = alt.Chart(df).mark_rule(color="black", strokeDash=[1,1]).encode(
    x="mean(bootstrap_means)"
)

pop_chart_bootstrap = pop_density_chart_bootstrap + pop_mean_chart_bootstrap

combined_chart = pop_chart_bootstrap + pickups_se_chart + pickups_t_chart

combined_chart


alt.LayerChart(...)

Compare these distributions: what do you notice? Write your answer below.

All three distributions are bell shaped and relatively symmetrical around the sample mean of ~39 pickups. Although the shape of the t distribution is nearly identical to the standard normal distribution, it is not quite the exact same shape, which might be on account of the small sample size(less than 30 students). As the sample size increases, the t distribution will more or less approach the standard normal distribution. Furthermore, because the t distribution has a higher kurtosis than the normal distribution - in other words, more weight in the tails, hence the "fatter" size of the tails - there is a higher probability that there will be extreme values that fall further away from the mean than the normal distribution.

# Part 2. Intervals, CDFs, and quantile dotplots (17 points)

Now that we are able to define the density, CDF, and quantile functions to describe our uncertainty in a mean, we can use these to construct various uncertainty visualizations as described in the lectures. For example, we can use the density function to create density plots or gradients; the CDF to create complementary CDF bar plots; and the quantile function to create intervals and quantile dotplots.

## 2.1 Intervals

### Question 2.1.1 Calculate 95% interval (2 points)

Using the quantile function (aka percentage points function) of the Student's t confidence distribution you derived in Part 1, calculate the lower and upper bounds of a 95% confidence interval for mean pickups and assign these bounds to the `pickups_lower` and `pickups_upper` variables. 

(Hint: use `stats.t.ppf()` as the quantile function and see the lecture on intervals.)

In [141]:
# YOUR CODE HERE
pickups_lower = stats.t.ppf(0.025, (len(pickups_df)-1), pickups_mean, pickups_se)
pickups_upper = stats.t.ppf(0.975, (len(pickups_df)-1), pickups_mean, pickups_se)



pickups_lower, pickups_upper

(36.50814451043221, 42.19185548956779)

In [142]:
dof = len(pickups_df) - 1
m = pickups_df['pickups'].mean()
se = stats.sem(pickups_df['pickups'])
#hiddent tests below

### Question 2.1.2 Visualize 95% + 66.666% interval (5 points)

Visualize a combined 95% + 66.666% (i.e. 2/3rds) interval for the mean of pickups, overlaid on top of the data (as ticks). 

Hints: 
- create two different layers, one for the 95% interval and one for the 66.666% interval, then combine them using `alt.layer()` or `+`.
- use `mark_rule(size=XXX).encode(x=YYY, x2=ZZZ)` to create each interval, where you fill in the values of `size` (to set the thickness of the interval), `x` (to set the lower bound), and `x2` (to set the upper bound).

Your output should look like this:

![Plot of 66% and 95% confidence intervals overliad on data](asset/assignment2_intervals.png)

In [143]:
# YOUR CODE HERE
pickups_lower_95 = pickups_lower
pickups_upper_95 = pickups_upper

level = 2/3
lower_quantile = (1 - level)/2
upper_quantile = (1 + level)/2


pickups_lower_66 = stats.t.ppf(lower_quantile, (len(pickups_df)-1), pickups_mean, pickups_se)
pickups_upper_66 = stats.t.ppf(upper_quantile, (len(pickups_df)-1), pickups_mean, pickups_se)

confidence_df = pd.DataFrame({"lower95": pickups_lower_95,"upper95":pickups_upper_95,"lower66":pickups_lower_66, "upper66": pickups_upper_66}, index=[0])


interval_chart_95 = alt.Chart(confidence_df).mark_rule(size=2).encode(
                x = alt.X('lower95', axis=alt.Axis(title="lower")),
                x2 = alt.X2('upper95'))

interval_chart_66 = alt.Chart(confidence_df).mark_rule(size=6).encode(
                x = alt.X('lower66', axis=alt.Axis(title="lower")),
                x2 = alt.X2('upper66'))


tick_chart = alt.Chart(pickups_df).mark_tick(
    color='red',
    thickness=1,
    size=40 * 0.3,  # controls width of tick.
).encode(
    x='pickups'
)
              



combined_interval_chart = tick_chart + interval_chart_95 + interval_chart_66 
combined_interval_chart

alt.LayerChart(...)

## 2.2 Cumulative distribution function

### Question 2.2.1 Visualize a Complementary CDF barplot (5 points)

Using the CDF function (or the CCDF function), plot a CCDF barplot for the Student's t-based confidence distribution. *The barplot should start at 0*.

Hint: start from the code for generating a density plot, above, and (1) adjust the code for generating `x` to ensure it includes 0 and (2) change the function from a PDF to 1 - the CDF.

Your output should look something like this:

![CCDF barplot of mean pickups](asset/assignment2_ccdf.png)

In [144]:
# YOUR CODE HERE


x = np.linspace(
    start = 0,
    stop = 55
    
)


cdf =  stats.t.cdf(x, (len(pickups_df)-1), pickups_mean, pickups_se)

cdf =  [1 -  K for K in cdf]

df = pd.DataFrame({"Mean pickups(CCDF)": x, "ccdf":  cdf})

pickups_cdf_chart = alt.Chart(df).mark_area(color='darkblue').encode(
    x=alt.X("Mean pickups(CCDF)", axis=alt.Axis(title="Mean pickups(CCDF)")),
    y="ccdf"
).properties(height=100, width=500)


pickups_cdf_chart

alt.Chart(...)

## 2.3 Quantile dotplots

Constructing quantile dotplots involves first generating a small-to-medium number of quantiles from the quantile function (aka percentage points function) of the uncertainty distribution. We do this by projecting evenly-spaced values in probability space back into the data units through the quantile function, then stacking up those values into a dotplot:

<img src="asset/cdf.png" alt="wo graphs stacked on top of each other. The top graph has a y-axis of p less than x, from 0.00 to 1.00 in intervals of 0.25, and the x-axis is x, from 0 to 30. The cumulative distribution function line starts out around (0, 0.00) and stays that way until about (8, 0.00) where it rises up the y axis until it reaches about (18, 1.00) and continues on to 30. The bottom graph has a y-axis of count, from 0.00 to 1.00 in intervals of 0.25, and the x-axis is x, from 0 to 30. Circles are stacked between 8 and 18 on the x-axis, with a peak around (11, 0.50) and is slightly skewed to the right." style="width: 500px;"/>

The first step is to generate the evenly-spaced probabilities. We will create a `ppoints(n)` function that generates `n` evenly-spaced probabilities (not including 0 and 1):

In [145]:
def ppoints(n):
    return (np.arange(n) + 0.5)/n

ppoints(20)

array([0.025, 0.075, 0.125, 0.175, 0.225, 0.275, 0.325, 0.375, 0.425,
       0.475, 0.525, 0.575, 0.625, 0.675, 0.725, 0.775, 0.825, 0.875,
       0.925, 0.975])

We can then use `ppoints` with the quantile function of the t confidence distribution (`stats.t.ppf`) to construct a quantile dotplot.

Dotplots in Altair are complex to construct, requiring three transformations:

1. `transform_bin` to bin the values
2. `transform_window` to rank the values within each bin (this stacks up all the dots in one bin)
3. `transform_joinaggregate` to calculate the midpoint of each bin using the median of all values in that bin

Putting together `ppoints`, the quantile function of the confidence distribution, and the above steps, we can construct a quantile dotplot.

In [146]:
df = pd.DataFrame({
    "x": stats.t.ppf(ppoints(50), len(pickups_df) - 1, pickups_mean, pickups_se)
})

alt.Chart(df).transform_bin(
    field="x",
    as_="bin",
    bin=alt.BinParams(step=0.75)
).transform_window(
    rank_in_bin="rank()",
    groupby=["bin"]
).transform_joinaggregate(
    bin_midpoint="median(x)",
    groupby=["bin"]
).mark_circle(size=90).encode(
    x="bin_midpoint:Q",
    y="rank_in_bin:Q"
).properties(width=600, height=100)

alt.Chart(...)

### Question 2.3.1 Construct a 20-dot quantile dotplot for the mean of pickups (5 points)

Construct a 20-dot quantile dotplot for the mean of pickups. Adjust the binning step size, circle size, and/or chart size to achieve a closely-packed quantile dotplot like the 50-dot dotplot above. It is typically also necessary to adjust the step size in the binning (`alt.BinParams(step=XXX)`), the size of the circles (`mark_circle(size=XXX)`) and/or the width and height of the plot (`properties(width=XXX, height=YYY)`) to achieve good-looking dotplots.

In [147]:
# YOUR CODE HERE
df = pd.DataFrame({
    "x": stats.t.ppf(ppoints(20), len(pickups_df) - 1, pickups_mean, pickups_se)
})

alt.Chart(df).transform_bin(
    field="x",
    as_="bin",
    bin=alt.BinParams(step=0.95)
).transform_window(
    rank_in_bin="rank()",
    groupby=["bin"]
).transform_joinaggregate(
    bin_midpoint="median(x)",
    groupby=["bin"]
).mark_circle(size=230).encode(
    x="bin_midpoint:Q",
    y="rank_in_bin:Q"
).properties(width=600, height=75)

alt.Chart(...)

## Part 3. Visualize world happiness (10 points)

We will do some visualization of the World Happiness Report dataset from 2015. It is a survey of the state of global happiness, which ranks 155 countries by their happiness levels. Let's explore the happiness score.

First, we'll read in the data and gather the happiness score for all countries:

In [148]:
#Read in 2015 World Happiness Report statistics as a dataframe
happ = pd.read_csv("asset/2015.csv")

#show the top 15 countries:
happ.head(15)

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176
5,Finland,Western Europe,6,7.406,0.03140,1.29025,1.31826,0.88911,0.64169,0.41372,0.23351,2.61955
6,Netherlands,Western Europe,7,7.378,0.02799,1.32944,1.28017,0.89284,0.61576,0.31814,0.47610,2.46570
7,Sweden,Western Europe,8,7.364,0.03157,1.33171,1.28907,0.91087,0.65980,0.43844,0.36262,2.37119
8,New Zealand,Australia and New Zealand,9,7.286,0.03371,1.25018,1.31967,0.90837,0.63938,0.42922,0.47501,2.26425
9,Australia,Australia and New Zealand,10,7.284,0.04083,1.33358,1.30923,0.93156,0.65124,0.35637,0.43562,2.26646


In [149]:
#some descriptive statistics of the World Happiness Report:
happ.describe()

,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
count,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000
mean,79.493671,5.375734,0.047885,0.846137,0.991046,0.630259,0.428615,0.143422,0.237296,2.098977
std,45.754363,1.145010,0.017146,0.403121,0.272369,0.247078,0.150693,0.120034,0.126685,0.553550
min,1.000000,2.839000,0.018480,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.328580
25%,40.250000,4.526000,0.037268,0.545808,0.856823,0.439185,0.328330,0.061675,0.150553,1.759410
50%,79.500000,5.232500,0.043940,0.910245,1.029510,0.696705,0.435515,0.107220,0.216130,2.095415
75%,118.750000,6.243750,0.052300,1.158448,1.214405,0.811013,0.549092,0.180255,0.309883,2.462415
max,158.000000,7.587000,0.136930,1.690420,1.402230,1.025250,0.669730,0.551910,0.795880,3.602140


## 3.1 Happiness Scores

We'll focus on two columns in particular: the `Happiness Score` (an aggregate happiness score for each country) and the `Standard Error`, which is the standard error of the `Happiness Score`: because the data are based on surveys, there is uncertainty in the measurement of the happiness score, and this is quantified using its standard error.

### Question 3.1.1 Visualize the happiness scores and uncertainty of the top 15 countries (5 points)

Use one of the uncertainty visualizations above to visualize the happiness score of each country. You will need to use the Normal approximation to the sampling distribution along with the standard error of the happiness score to do this.

You **MUST NOT** use just a combination of a point and a single interval to visualize the scores. For example, the following chart **IS NOT AN ACCEPTABLE SOLUTION** (but could be a good place to start):

![point estimates and 95% confidence intervals for the happiness score in the top 10 countries according to the World Happiness Report](asset/assignment2_happiness_intervals.png)

But you could construct a chart with a similar layout, but which uses one of the uncertainty encodings you learned above. For example, you might start by constructing a chart with 95% intervals, then extend it to be a chart with 95%+66.666% intervals, which would be an acceptable solution.

Hints:

1. You may want to sort the y axis by happiness score using something like `alt.Y("Country", sort=["Happiness Score"]`
2. You may want to restrict the x axis domain to make it easier to see the uncertainty using something like `alt.X("Happiness Score", scale={"domain":[6,8]})`
3. If you decide to make density plots, quantile dotplots, or CCDF barplots, you need to use the `facet()` function to display multiple subcharts where each one is a single country. If you do that, you may want to set the `"labelAngle"` property on the facet headers so they are horizontal.

In [150]:
# YOUR CODE HERE
happ['se'] = stats.sem(happ['Happiness Score'])

happ["lower95"]=stats.norm.ppf(0.025,happ['Happiness Score'],happ['se'])
happ["upper95"]=stats.norm.ppf(0.975,happ['Happiness Score'],happ['se'])


level = 2/3
lower_quantile = (1 - level)/2
upper_quantile = (1 + level)/2

happ["lower66"]=stats.norm.ppf(lower_quantile,happ['Happiness Score'],happ['se'])
happ["upper66"]=stats.norm.ppf(upper_quantile,happ['Happiness Score'],happ['se'])

happ_copy = happ

happ_copy =  happ_copy.sort_values(['Happiness Score'], ascending=[False])

happ_copy = happ_copy.head(15)


interval_chart_95 = alt.Chart(happ_copy).mark_rule().encode(
                y = alt.Y("Country", sort=["Happiness Score"]),
                x = alt.X("lower95", axis=alt.Axis(title="Happiness Score, lower, upper"), scale={"domain":[6,8]}),
                x2 = alt.X2("upper95"))

interval_chart_66 = alt.Chart(happ_copy).mark_rule(size=3).encode(
                y = alt.Y("Country", sort=["Happiness Score"]),
                x = alt.X("lower66", axis=alt.Axis(title="Happiness Score, lower, upper"), scale={"domain":[6,8]}),
                x2 = alt.X2("upper66"))

point_chart = alt.Chart(happ_copy).mark_circle(color="red",size =30).encode(
                y = alt.Y("Country", sort=["Happiness Score"]),
                x = alt.X("Happiness Score", scale={"domain":[6,8]}))


combined_country_chart = interval_chart_95 + interval_chart_66 + point_chart


combined_country_chart


alt.LayerChart(...)

### Question 3.1.2 Reflect on the chart you created (5 points)

What are the pros and cons of the uncertainty encoding you used? How might it compare to other options you could have chosen?

I believe that the interval chart above holds some advantages over an alternative density chart representation for the reason that, in general, audiences have an easier time judging probability based on position as opposed to area, and having a good sense of area perception is integral to interpreting a density plot accurately. A viewer may interpret a shaded region as containing a larger portion of the "whole pie" that it actually has. Furthermore, a single, more condensed chart, as opposed to a faceted small-multiples chart that would have been necessary if pursuing a density plot approach, helps to facilitate easier comparison between multiple countries. 


Even so, there are issues associated with interval representations, in the sense that distributional information is absent. Visual information such as the skewness or kurtosis of the distrubution for each country is not available within the chart above. In order to effectively gauge how symmetrical the distribution is, the viewer would need to eyeball the placement of the red dot(happiness score), which is somewhat difficult. Furthermore, it is possible for the viewer to erroneously interpret the interval as absolutely containing the real-world data point. Though the addition of multiple intervals(66% and 95%) may help drive home the fact that neither interval is unimpeachable, an untrained viewer may very likely interpret the interval in such a manner.

Please remember to submit both the HTML and .ipynb formats of your completed notebook. When generating your HTML, be sure to run your complete code first before downloading as HTML. Please remember to work on your explanations and interpretations!